In [ ]:
#!/usr/bin/env python
# coding: utf-8

# Firm characteristics download from WRDS
# This code illustrates how to get COMPUSTAT, CRSP and S&P500 index constituents, with their identifiers of GVKEY and PERMNO.

# The data for computing the firm characteristics should be within the realm of the datasets of COMPUSTAT and CRSP.

# Authors: Jiacheng Zou (jiachengzou@stanford.edu), Dehan Cui (dc3769@columbia.edu)
# Date: Oct 12, 2024

# can be run with Linux nohup command

import pandas as pd
import numpy as np
from pandas.tseries.offsets import *
import pickle as pkl
import pyarrow.feather as feather
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
from google.colab import drive
import os
drive.mount('/content/drive')
storage_folder = '/content/drive/MyDrive/data'

os.chdir(storage_folder)

Mounted at /content/drive


In [ ]:
'''
The files in storage_folder include:
	Beta_fp.csv
	Beta_ln.csv
	crsp_mon.csv
	crsp_daily.csv
	Idio_vol.csv
	compustat_a.csv
	compustat_q.csv
	ff_daily.csv
	ff_mon.csv
'''

'\nThe files in storage_folder include:\n\tBeta_fp.csv\n\tBeta_ln.csv\n\tcrsp_mon.csv\n\tcrsp_daily.csv\n\tIdio_vol.csv\n\tcompustat_a.csv\n\tcompustat_q.csv\n\tff_daily.csv\n\tff_mon.csv\n'

In [ ]:
# Step 1: CRSP
# First we get monthly return data from CRSP, and then align the rest of the data with CRSP.

# The Center for Research in Security Prices, LLC (CRSP) maintains the most comprehensive collection of security price, return, and volume data for the NYSE, AMEX and NASDAQ stock markets. Additional CRSP files provide stock indices, beta-based and cap-based portfolios, treasury bond and risk-free rates, mutual funds, and real estate data.

# We use the data table here: https://wrds-www.wharton.upenn.edu/pages/get-data/center-research-security-prices-crsp/quarterly-update/stock-version-2/monthly-stock-file/

# 'primaryexch':'N' - NYSE, 'A' - Amex (NYSE American, a small cap equity market), 'Q' - 'Nasdaq'

# Set beginning dates for CRSP and compustat tables:
beginning_date_crsp = "197807" # specify the beginning date of return data from crsp, in the format of yyyymm
beginning_date_compustat = "06/01/1976" # to be safe, we specify the beginning date of accounting data, which should be 2yr before crsp due to lagging of accounting data

In [ ]:
# Read the monthly stock file
crsp_mon = pd.read_csv('crsp_mon.csv')
# Convert all the column names to lowercase for convenience
crsp_mon.columns = crsp_mon.columns.str.lower()

# Drop duplicate rows and keep the last occurences
checking_cols=['primaryexch','yyyymm','permno','permco','ticker','mthprc','shrout']
crsp_mon = crsp_mon.drop_duplicates(subset=checking_cols,keep='last')

# Change the data type of certain columns to int:
crsp_mon[['permno','yyyymm']] = crsp_mon[['permno','yyyymm']].astype(int)

crsp_mon = crsp_mon.dropna(subset=['mthprc'])
crsp_mon['me'] = crsp_mon['mthprc'].abs() * crsp_mon['shrout']  # calculate market equity

# If Market Equity is Nan then let return equals to 0:
crsp_mon.loc[crsp_mon['me'].isna(), 'mthret'] = 0.0

# Select variables to prepare merging with compustat:
crsp_mon.drop_duplicates(inplace=True)

# Sort the table and reset the index:
crsp_mon.sort_values(['permno','yyyymm'],inplace=True)
crsp_mon.reset_index(drop=True,inplace=True)

<ipython-input-8-2058347c4793>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp_mon[['permno','yyyymm']] = crsp_mon[['permno','yyyymm']].astype(int)
<ipython-input-8-2058347c4793>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp_mon['me'] = crsp_mon['mthprc'].abs() * crsp_mon['shrout']  # calculate market equity


In [ ]:
# Read the daily stock file
crsp_daily = pd.read_csv('crsp_daily.csv')
# Convert all the column names to lowercase for convenience
crsp_daily.columns = crsp_daily.columns.str.lower()

# Handle duplicate mixed type columns
crsp_daily[['hdrcusip','hdrcusip9','cusip','cusip9']] = crsp_daily[['hdrcusip','hdrcusip9','cusip','cusip9']].astype(str)

# Drop duplicate rows and keep the last occurences
dly_checking_cols=['primaryexch','yyyymmdd','permno','permco','ticker','dlyprc','shrout']
crsp_daily = crsp_daily.drop_duplicates(subset=dly_checking_cols,keep='last')

# Change the data type of certain columns to int:
crsp_daily[['permno','yyyymmdd']] = crsp_daily[['permno','yyyymmdd']].astype(int)

crsp_daily.sort_values(['permno','yyyymmdd'],inplace=True)
crsp_daily.reset_index(drop=True,inplace=True)
crsp_daily_chunk = crsp_daily.copy()

<ipython-input-9-ba7f5f539cde>:2: DtypeWarning: Columns (1,2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  crsp_daily = pd.read_csv('crsp_daily.csv')


In [ ]:
# Read the Fama-French files
ff_data_mon = pd.read_csv('ff_mon.csv')
ff_data_mon['yyyymm']= ff_data_mon['dateff'] // 100
ff_data_mon.sort_values(['yyyymm'],inplace=True)
ff_data_mon.reset_index(drop=True,inplace=True)

ff_data_daily = pd.read_csv('ff_daily.csv')
ff_data_daily_chunk=ff_data_daily[['date','rf','mktrf','smb','hml','umd']]
ff_data_daily_chunk['yyyymmdd'] = ff_data_daily_chunk['date']
ff_data_daily_chunk.sort_values(['yyyymmdd'],inplace=True)
ff_data_daily_chunk.reset_index(drop=True,inplace=True)
ff_data_daily_chunk['lag_mktrf'] = ff_data_daily_chunk['mktrf'].shift(1).bfill()

<ipython-input-10-7acd5fc66c49>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ff_data_daily_chunk['yyyymmdd'] = ff_data_daily_chunk['date']
<ipython-input-10-7acd5fc66c49>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ff_data_daily_chunk.sort_values(['yyyymmdd'],inplace=True)
<ipython-input-10-7acd5fc66c49>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

In [ ]:
crsp_daily_chunk.sort_values(['yyyymmdd','permno'],inplace=True)
ff_data_daily_chunk.sort_values(['yyyymmdd'],inplace=True)

# Merge the two dataframes for future use:
CRSP_daily_with_rf=pd.merge(crsp_daily_chunk,ff_data_daily_chunk,on='yyyymmdd',how='left')
CRSP_daily_with_rf['excess_ret']=CRSP_daily_with_rf['dlyret']-CRSP_daily_with_rf['rf']
CRSP_daily_with_rf = CRSP_daily_with_rf.sort_values(['permno','yyyymmdd']).reset_index(drop=True)

<ipython-input-11-b58adfac0843>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ff_data_daily_chunk.sort_values(['yyyymmdd'],inplace=True)


In [ ]:
# Create functions for monthly, quarterly, double-quarterly, annual, and WRDS laggings:
def lag_yyyymm_columns(df, yyyymm_col):
    """
    Add lagged YYYYMM columns for monthly, quarterly, and annual data to the DataFrame.

    Parameters:
    df (DataFrame): The input DataFrame.
    yyyymm_col (str): The name of the YYYYMM column in the DataFrame.

    Returns:
    DataFrame: The DataFrame with new columns for lagged YYYYMM values.
    """

    # Monthly lagging
    def lag_monthly(yyyymm):
        year = yyyymm // 100
        month = yyyymm % 100
        month -= 1
        if month <= 0:
            month += 12
            year -= 1

        return year * 100 + month

    # Quarterly lagging
    def lag_quarterly(yyyymm):
        year = yyyymm // 100
        month = yyyymm % 100
        month -= 5
        if month <= 0:
            month += 12
            year -= 1

        return year * 100 + month

    # Annual lagging
    def lag_annual(yyyymm):
        year = yyyymm // 100
        month = yyyymm % 100
        month -= 7
        if month <= 0:
            month += 12
            year -= 1
        return year * 100 + month

    # WRDS lagging
    def lag_wrds(yyyymm):
        year = yyyymm // 100
        month = yyyymm % 100
        month -= 3
        if month <= 0:
            month += 12
            year -= 1
        return year * 100 + month

    # Add lagging columns to existing DataFrame
    df['yyyymm_l_mon'] = df[yyyymm_col].apply(lag_monthly)
    df['yyyymm_l_q'] = df[yyyymm_col].apply(lag_quarterly)
    df['yyyymm_l_a'] = df[yyyymm_col].apply(lag_annual)
    df['yyyymm_l_wrds'] = df[yyyymm_col].apply(lag_wrds)

    return df

def lag_monthly(yyyymm):
        year = yyyymm // 100
        month = yyyymm % 100
        month -= 1
        if month <= 0:
            month += 12
            year -= 1

        return year * 100 + month

In [ ]:
def create_one_more_lag(df, compute_variables):
    # Sort by 'gvkey' and 'yyyymm'
    df.sort_values(['gvkey', 'yyyymm'], inplace=True)

    # Generate lag variable names
    lag_names = ['lag_' + var for var in compute_variables]

    # Compute lagged variables
    df[lag_names] = df.groupby('gvkey', as_index=False)[compute_variables].shift()

    # Sort by 'yyyymm' and 'gvkey'
    df.sort_values(['yyyymm', 'gvkey'], inplace=True)

    # Fill NA values in lagged variables with original values
    for i, lag_name in enumerate(lag_names):
        df.loc[df[lag_name].isna(), lag_name] = df.loc[df[lag_name].isna(), compute_variables[i]]

    return df


# Impute the missing values
def impute_missing_var(df: pd.DataFrame, method: str = 'zero',
                       identifiers: list = ['sic2','yyyymm'], compute_variables: list = ['at']) -> pd.DataFrame:
    """
    Impute missing values in the specified columns of a DataFrame.

    Parameters:
    df (pd.DataFrame): The input DataFrame containing data to impute.
    method (str): The imputation method. Possible values are 'xsection' or 'zero'.
                  'xsection' - Impute by sic2 cross-sectional median, then by 0.
                  'zero' - Impute by 0.

    Returns:
    pd.DataFrame: The DataFrame with missing values imputed.

    Raises:
    ValueError: If the method is not one of 'xsection' or 'zero'.
    """
    if method not in ['xsection', 'zero']:
        raise ValueError("Invalid method. Use 'xsection' or 'zero'.")

    if method == 'xsection':
        # Calculate cross-sectional median of last year
        df['yyyy_lag1']=df['yyyymm']//100-1
        cross_sectional_median = df.groupby(['sic2','yyyy_lag1'],as_index=False)[compute_variables].median()

        # Fill missing values with the cross-sectional median
        df[compute_variables] = df[compute_variables].fillna(cross_sectional_median)
    else:
        # Fill any remaining missing values with zero, regardless of method argument
        df[compute_variables] = df[compute_variables].fillna(0)

    return df

In [ ]:
# Read quarterly and annual fundamental data
compustat_q = pd.read_csv('compustat_q.csv')
compustat_a = pd.read_csv('compustat_a.csv')
# Convert datadate to yyyymm format
compustat_q['yyyymm'] = compustat_q['datadate'] // 100
compustat_a['yyyymm'] = compustat_a['datadate'] // 100

# sort and clean up:
compustat_q = compustat_q.sort_values(by=['yyyymm','cusip']).drop_duplicates()
compustat_a = compustat_a.sort_values(by=['yyyymm','cusip']).drop_duplicates()

# Reset index
compustat_q.reset_index(drop=True,inplace=True)
compustat_a.reset_index(drop=True,inplace=True)

In [ ]:
# Use the lagging function on CRSP_kept table to create lagged columns:
CRSP_kept = lag_yyyymm_columns(crsp_mon,'yyyymm')

# Prepare quarterly data table merging:
CRSP_kept_q = CRSP_kept.sort_values(['yyyymm_l_q','cusip9','yyyymm'])
CRSP_kept_q['common_cusip'] = CRSP_kept_q['cusip9'].astype(str)
compustat_q.sort_values(['yyyymm','cusip'],inplace=True)
compustat_q['common_cusip'] = compustat_q['cusip'].astype(str)

# Merging by the as-of logic:
merged_with_q=pd.merge_asof(CRSP_kept_q,compustat_q,by='common_cusip',
                            left_on='yyyymm_l_q',right_on='yyyymm',suffixes=('','_comp_q'),direction='backward')
merged_with_q_wrds_rule = pd.merge_asof(CRSP_kept_q,compustat_q,by='common_cusip',
                                        left_on='yyyymm_l_wrds',right_on='yyyymm',suffixes=('','_comp_q'),direction='backward')

In [ ]:
# Prepare annual data table merging:
merged_with_q.sort_values(['yyyymm_l_a','cusip9','yyyymm'],inplace=True)
merged_with_q['common_cusip'] = merged_with_q['cusip9'].astype(str)
compustat_a.sort_values(['yyyymm','cusip'],inplace=True)
compustat_a['common_cusip'] = compustat_a['cusip'].astype(str)

# Merge the tables:
merged_with_a_q=pd.merge_asof(merged_with_q,compustat_a,by='common_cusip',
                              left_on='yyyymm_l_a',right_on='yyyymm',suffixes=('','_comp_a'),direction='backward')
merged_with_a_q_wrds_rule=pd.merge_asof(merged_with_q_wrds_rule,compustat_a,by='common_cusip',
                                        left_on='yyyymm_l_wrds',right_on='yyyymm',suffixes=('','_comp_a'),direction='backward')

In [ ]:
# Merge with market return and risk-free rate
merged_with_a_q_r=pd.merge_asof(merged_with_a_q,ff_data_mon,
                                by='yyyymm',on='yyyymm',direction='backward')
merged_with_a_q_r_wrds_rule=pd.merge_asof(merged_with_a_q_wrds_rule,ff_data_mon,
                                          by='yyyymm',on='yyyymm',direction='backward')

In [ ]:
merged_with_a_q_r = merged_with_a_q_r.dropna(subset=['gvkey'])
merged_with_a_q_r_wrds_rule = merged_with_a_q_r_wrds_rule.dropna(subset=['gvkey'])

In [ ]:
# Calculate excess returns:
merged_with_a_q_r['excret'] = merged_with_a_q_r['mthret'] - merged_with_a_q_r['rf']
merged_with_a_q_r['mkt_excret'] = merged_with_a_q_r['sprtrn'] - merged_with_a_q_r['rf']
merged_with_a_q_r_wrds_rule['excret'] = merged_with_a_q_r_wrds_rule['mthret'] - merged_with_a_q_r_wrds_rule['rf']
merged_with_a_q_r_wrds_rule['mkt_excret'] = merged_with_a_q_r_wrds_rule['sprtrn'] - merged_with_a_q_r_wrds_rule['rf']

# Create sic2:
merged_with_a_q_r['sic2'] = merged_with_a_q_r['sic'].astype(str).str[:2]
merged_with_a_q_r_wrds_rule['sic2'] = merged_with_a_q_r_wrds_rule['sic'].astype(str).str[:2]

<ipython-input-19-d49f54db005d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_with_a_q_r_wrds_rule['excret'] = merged_with_a_q_r_wrds_rule['mthret'] - merged_with_a_q_r_wrds_rule['rf']
<ipython-input-19-d49f54db005d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_with_a_q_r_wrds_rule['mkt_excret'] = merged_with_a_q_r_wrds_rule['sprtrn'] - merged_with_a_q_r_wrds_rule['rf']


In [ ]:
# Fill all the missing fundamental variables (quarterly and annual frequency) by industry median:
fundamental_vars = ['atq','actq','cheq','lctq','dlcq','txpq','dpq','saleq',
                    'ivaoq','dlttq','mibq','pstkq','ceqq','seqq','ltq','txditcq',
                    'ibq','cogsq','cshoq','ajexq','ppegtq','invtq','wcapq','niq',
                    'piq','xsgaq','oiadpq','txdbq','mkvaltq','nopiq','rectq','ppentq',
                    'at','ivao','dvc','prstkc','pstkrv','sstk','wcap','wcapch','capx',
                    'pi','gp','ebit','sale']

merged_with_a_q_r=impute_missing_var(merged_with_a_q_r,
                                     method='xsection',
                                     compute_variables=fundamental_vars)
merged_with_a_q_r_wrds_rule=impute_missing_var(merged_with_a_q_r_wrds_rule,
                                               method='xsection',
                                               compute_variables=fundamental_vars)

merged_with_a_q_r.sort_values(['permno','yyyymm'],inplace=True)
merged_with_a_q_r_wrds_rule.sort_values(['permno','yyyymm'],inplace=True)

In [ ]:
# Start the calculation of features


In [ ]:
# Total Assets (AT)
CRSP_at = merged_with_a_q_r.copy()
CRSP_at.sort_values(['permno','yyyymm'],inplace=True)

CRSP_at['AT'] = CRSP_at['atq']
CRSP_at['AT'] = CRSP_at['AT'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# Assets-to-Market Cap (A2ME)
# Formula: A2ME = AT / (PRC*SHROUT)
CRSP_a2me = CRSP_at.copy()

CRSP_a2me = create_one_more_lag(CRSP_a2me,['shrout','mthprc'])
CRSP_a2me['A2ME'] = CRSP_a2me['atq'] / (CRSP_a2me['lag_shrout']*CRSP_a2me['lag_mthprc'])
CRSP_a2me['A2ME'] = CRSP_a2me['A2ME'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# Absolute Value of Operating Accruals (AOA)
# Formula: AOA = |[delta(ACT-CHE-(LCT+DLC+TXP))-DP] / lagged_AT|
CRSP_aoa = CRSP_a2me.copy()
CRSP_aoa = create_one_more_lag(CRSP_aoa, ['atq'])

# Step 1: Non_Cash_Current_Assets = ACT - CHE
CRSP_aoa['non_cash_CA'] = CRSP_aoa['actq']-CRSP_aoa['cheq']

# Step 2: Non_Cash_Working_Capital = (Non_Cash_Current_Assets - (LCT + DLC + TXP)
CRSP_aoa['non_cash_WC'] = CRSP_aoa['non_cash_CA']-(CRSP_aoa['lctq']+CRSP_aoa['dlcq']+CRSP_aoa['txpq'])

# Step 3: Calculate the changes in non-cash working capital
CRSP_aoa.sort_values(['permno','yyyymm'],inplace=True)
CRSP_aoa['non_cash_WC_delta'] = CRSP_aoa.groupby('permno')[['non_cash_WC']].diff()
CRSP_aoa['non_cash_WC_delta'] = CRSP_aoa['non_cash_WC_delta'].replace(0.0, np.nan)
CRSP_aoa['non_cash_WC_delta'] = CRSP_aoa.groupby('permno')[['non_cash_WC_delta']].ffill()
CRSP_aoa['non_cash_WC_delta'] = CRSP_aoa['non_cash_WC_delta'].fillna(0.0)

# Step 4: OA = [delta(Non_Cash_Working_Capital) - DP] / lagged AT
oa = (CRSP_aoa['non_cash_WC_delta']-CRSP_aoa['dpq']) / CRSP_aoa['lag_atq']

# Step 5: Take the absolute value of OA (AOA = |OA|)
CRSP_aoa['AOA'] = oa.abs()
CRSP_aoa['AOA'] = CRSP_aoa['AOA'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# Net Sales over Lagged Net Operating Assets (ATO)
# Formula (All the term except net_sales here are lagged):
# ATO = net_sales / (AT-CHE-IVAO-(AT-DLC-DLTT-MIB-PSTK-CEQ))
CRSP_ato = CRSP_aoa.copy()

# Fill nan values in IVAO with 0:
CRSP_ato = impute_missing_var(CRSP_ato,
                              method='zero',
                              compute_variables=['ivaoq'])

# Create required lagged variables:
ato_vars = ['saleq','atq','cheq','ivaoq','dlcq','dlttq','mibq','pstkq','ceqq']
CRSP_ato = create_one_more_lag(CRSP_ato, ato_vars)

# net_sales = SALE
net_sales = CRSP_ato['saleq']
# lagged_operating_assets = lagged_AT - lagged_CHE - lagged_IVAO
lag_operating_assets = CRSP_ato['lag_atq']-CRSP_ato['lag_cheq']-CRSP_ato['lag_ivaoq']
# lagged_operating_liabilities = lagged_AT - lagged_DLC - lagged_DLTT - lagged_MIB - lagged_PSTK - lagged_CEQ
lag_operating_liabs = CRSP_ato['lag_atq']-CRSP_ato['lag_dlcq']-CRSP_ato['lag_dlttq']-CRSP_ato['lag_mibq']-CRSP_ato['lag_pstkq']-CRSP_ato['lag_ceqq']

# ATO = SALE / (lagged_operating_assets-lagged_operating_liabilities)
CRSP_ato['ATO'] = net_sales / (lag_operating_assets-lag_operating_liabs)
CRSP_ato['ATO'] = CRSP_ato['ATO'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# Book-to-Market Equity (BEME)
# Formula: BEME = (AT-LT) / (SHROUT*PRC)
CRSP_beme = CRSP_ato.copy()

# Shareholder Equity
CRSP_beme['SH'] = CRSP_beme['seqq']
CRSP_beme.loc[CRSP_beme['SH'].isna(), 'SH'] = CRSP_beme.loc[CRSP_beme['SH'].isna(), 'ceqq']+CRSP_beme.loc[CRSP_beme['SH'].isna(), 'pstkq']
CRSP_beme.loc[CRSP_beme['SH'].isna(), 'SH'] = CRSP_beme.loc[CRSP_beme['SH'].isna(), 'atq']-CRSP_beme.loc[CRSP_beme['SH'].isna(), 'ltq']
# book_value_of_equity
CRSP_beme['BV'] = CRSP_beme['SH'] + CRSP_beme['txditcq'] - CRSP_beme['pstkq']

# BEME = book_value_of_equity / (SHROUT*PRC)
CRSP_beme['BEME'] = CRSP_beme['BV'] / (CRSP_beme['lag_shrout']*CRSP_beme['lag_mthprc'])
CRSP_beme['BEME'] = CRSP_beme['BEME'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# Adjusted Book-to-Market Equity (BEME_adj)
# Calculate the industry average for each industry at each time point
CRSP_beme_adj = CRSP_beme.copy()
CRSP_beme_adj.sort_values(['yyyymm','sic2'],inplace=True)
beme_mean = CRSP_beme_adj.groupby(['yyyymm','sic2'],as_index=False)['BEME'].mean()
beme_adj_merged = pd.merge(CRSP_beme_adj, beme_mean, on=['yyyymm','sic2'],how='left',suffixes=('', '_mean'))

beme_adj_merged.sort_values(['permno','yyyymm'],inplace=True)
beme_adj_merged['BEME_adj'] = beme_adj_merged['BEME'] - beme_adj_merged['BEME_mean']
beme_adj_merged['BEME_adj'] = beme_adj_merged['BEME_adj'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# Beta
# Beta Calculation under Frazzini and Pedersen (2014)
beta_fp = pd.read_csv('Beta_fp.csv')
beta_fp.columns = beta_fp.columns.str.lower()
beta_fp.sort_values(['permno','date'],inplace=True)

beta_fp['yyyymmdd'] =  pd.to_datetime(beta_fp['date'].astype(str), format='%Y%m%d')
beta_fp.sort_values(['permno','yyyymmdd'],inplace=True)
beta_fp.set_index('yyyymmdd', inplace=True)

resample_beta_fp = beta_fp.groupby('permno').resample('M').last().drop(columns=['permno'])
resample_beta_fp.reset_index(inplace=True)
resample_beta_fp['yyyymmdd'] = resample_beta_fp['yyyymmdd'].dt.strftime('%Y%m%d').astype(int)
resample_beta_fp['yyyymm'] = resample_beta_fp['yyyymmdd'] // 100

resample_beta_fp.sort_values(['yyyymm','permno'],inplace=True)
beme_adj_merged.sort_values(['yyyymm_l_mon','permno'],inplace=True)
CRSP_beta_fp = pd.merge_asof(beme_adj_merged,resample_beta_fp[['yyyymm','permno','b_mkt']],by='permno',
                             left_on='yyyymm_l_mon',right_on='yyyymm',suffixes=('','_beta_fp'),direction='backward')
CRSP_beta_fp.rename(columns={'b_mkt':'Beta_FP'},inplace=True)

CRSP_beta_fp['Beta_FP'] = CRSP_beta_fp['Beta_FP'].replace([np.inf, -np.inf], np.nan)

In [ ]:
CRSP_beta_fp.to_csv('beta_fp_finished.csv')

In [ ]:
# Beta Calculation under Lewellen and Nagel (2006)
beta_ln = pd.read_csv('Beta_ln.csv')
beta_ln.columns = beta_ln.columns.str.lower()
beta_ln.sort_values(['permno','date'],inplace=True)

beta_ln['yyyymmdd'] =  pd.to_datetime(beta_ln['date'].astype(str), format='%Y%m%d')
beta_ln.sort_values(['permno','yyyymmdd'],inplace=True)
beta_ln.set_index('yyyymmdd', inplace=True)

resample_beta_ln = beta_ln.groupby('permno').resample('M').last().drop(columns=['permno'])
resample_beta_ln.reset_index(inplace=True)
resample_beta_ln['yyyymmdd'] = resample_beta_ln['yyyymmdd'].dt.strftime('%Y%m%d').astype(int)
resample_beta_ln['yyyymm'] = resample_beta_ln['yyyymmdd'] // 100

resample_beta_ln.sort_values(['yyyymm','permno'],inplace=True)
CRSP_beta_fp.sort_values(['yyyymm_l_mon','permno'],inplace=True)
CRSP_beta_ln = pd.merge_asof(CRSP_beta_fp,resample_beta_ln[['yyyymm','permno','b_mkt']],by='permno',
                             left_on='yyyymm_l_mon',right_on='yyyymm',suffixes=('','_beta_ln'),direction='backward')
CRSP_beta_ln.rename(columns={'b_mkt':'Beta_LN'},inplace=True)

CRSP_beta_ln['Beta_LN'] = CRSP_beta_ln['Beta_LN'].replace([np.inf, -np.inf], np.nan)

In [ ]:
CRSP_beta_ln.to_csv('beta_ln_finished.csv')

In [ ]:
# Beta_daily
# Define a function to perform the rolling regression
def rolling_regression(group):
    # Check if the group has enough data points
    if len(group) < 21 or all(group['excess_ret'].isna()):
        return pd.Series([0] * len(group))

    rolling_model = RollingOLS(group['excess_ret'], sm.add_constant(group[['mktrf','lag_mktrf']]), window=21, min_nobs=15).fit()
    return rolling_model.params['mktrf']+rolling_model.params['lag_mktrf']

# Beta_daily Calculation
beta_daily = CRSP_daily_with_rf.copy()

beta_daily.sort_values(['permno','yyyymmdd'],inplace=True)
beta_daily['Beta_daily'] = beta_daily.groupby('permno').apply(rolling_regression).reset_index(drop=True)

beta_daily['yyyymmdd'] =  pd.to_datetime(beta_daily['yyyymmdd'].astype(str), format='%Y%m%d')
beta_daily.sort_values(['permno','yyyymmdd'],inplace=True)
beta_daily.set_index('yyyymmdd', inplace=True)
beta_monthly = beta_daily.groupby('permno').resample('M').last().drop(columns=['permno']).reset_index()
beta_monthly['yyyymmdd'] = beta_monthly['yyyymmdd'].dt.strftime('%Y%m%d').astype(int)
beta_monthly['yyyymm'] = beta_monthly['yyyymmdd'] // 100

beta_monthly.sort_values(['yyyymm','permno'],inplace=True)
CRSP_beta_ln.sort_values(['yyyymm_l_mon','permno'],inplace=True)
CRSP_beta_daily = pd.merge_asof(CRSP_beta_ln,beta_monthly[['yyyymm','permno','Beta_daily']],by='permno',
                             left_on='yyyymm_l_mon',right_on='yyyymm',suffixes=('','_beta_daily'),direction='backward')

CRSP_beta_daily['Beta_daily'] = CRSP_beta_daily['Beta_daily'].replace([np.inf, -np.inf], np.nan)

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/rolling.py:272: RuntimeWarning: divide by zero encountered in log
  llf = -np.log(ssr) * nobs2  # concentrated likelihood
/usr/local/lib/python3.10/dist-packages/statsmodels/regression/rolling.py:272: RuntimeWarning: divide by zero encountered in log
  llf = -np.log(ssr) * nobs2  # concentrated likelihood
/usr/local/lib/python3.10/dist-packages/statsmodels/regression/rolling.py:272: RuntimeWarning: divide by zero encountered in log
  llf = -np.log(ssr) * nobs2  # concentrated likelihood
/usr/local/lib/python3.10/dist-packages/statsmodels/regression/rolling.py:272: RuntimeWarning: divide by zero encountered in log
  llf = -np.log(ssr) * nobs2  # concentrated likelihood
/usr/local/lib/python3.10/dist-packages/statsmodels/regression/rolling.py:272: RuntimeWarning: divide by zero encountered in log
  llf = -np.log(ssr) * nobs2  # concentrated likelihood
/usr/local/lib/python3.10/dist-packages/statsmodels/regression/rolling.py:2

In [ ]:
CRSP_beta_daily.to_csv('beta_daily_finished.csv')

In [ ]:
# C
CRSP_C = CRSP_beta_daily.copy()

CRSP_C['C'] = CRSP_C['cheq'] / CRSP_C['atq']
CRSP_C['C'] = CRSP_C['C'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# C2D
CRSP_C2D = CRSP_C.copy()

CRSP_C2D['C2D'] = (CRSP_C2D['ibq']+CRSP_C2D['dpq']) / CRSP_C2D['ltq']
CRSP_C2D['C2D'] = CRSP_C2D['C2D'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# CTO
CRSP_CTO = CRSP_C2D.copy()

CRSP_CTO['CTO'] = CRSP_CTO['saleq'] / CRSP_CTO['lag_atq']
CRSP_CTO['CTO'] = CRSP_CTO['CTO'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# Debt2P
CRSP_d2p = CRSP_CTO.copy()

CRSP_d2p['Debt2P'] = (CRSP_d2p['dlttq']+CRSP_d2p['dlcq']) / (CRSP_d2p['lag_shrout']*CRSP_d2p['lag_mthprc'])
CRSP_d2p['Debt2P'] = CRSP_d2p['Debt2P'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# Delta_ceq
CRSP_delta_ceq = CRSP_d2p.copy()
CRSP_delta_ceq.sort_values(['permno','yyyymm'],inplace=True)

CRSP_delta_ceq['delta_ceq'] = CRSP_delta_ceq.groupby('permno')[['ceqq']].pct_change()
CRSP_delta_ceq['delta_ceq'] = CRSP_delta_ceq['delta_ceq'].replace(0.0, np.nan)
CRSP_delta_ceq['delta_ceq'] = CRSP_delta_ceq.groupby('permno')[['delta_ceq']].ffill()
CRSP_delta_ceq['delta_ceq'] = CRSP_delta_ceq['delta_ceq'].fillna(0.0)

CRSP_delta_ceq['delta_ceq'] = CRSP_delta_ceq['delta_ceq'].replace([np.inf, -np.inf], np.nan)

<ipython-input-48-8c496656b9a6>:5: FutureWarning: The default fill_method='ffill' in DataFrameGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  CRSP_delta_ceq['delta_ceq'] = CRSP_delta_ceq.groupby('permno')[['ceqq']].pct_change()


In [ ]:
#Delta(delta_gm - delta_sales)
CRSP_del_delgm_delsale = CRSP_delta_ceq.copy()

CRSP_del_delgm_delsale['gross_margin'] = CRSP_del_delgm_delsale['saleq']-CRSP_del_delgm_delsale['cogsq']

CRSP_del_delgm_delsale['delta_gm'] = CRSP_del_delgm_delsale.groupby('permno')[['gross_margin']].pct_change()
CRSP_del_delgm_delsale['delta_gm'] = CRSP_del_delgm_delsale['delta_gm'].replace(0.0, np.nan)
CRSP_del_delgm_delsale['delta_gm'] = CRSP_del_delgm_delsale.groupby('permno')[['delta_gm']].ffill()
CRSP_del_delgm_delsale['delta_gm'] = CRSP_del_delgm_delsale['delta_gm'].fillna(0.0)

CRSP_del_delgm_delsale['delta_sales'] = CRSP_del_delgm_delsale.groupby('permno')[['saleq']].pct_change()
CRSP_del_delgm_delsale['delta_sales'] = CRSP_del_delgm_delsale['delta_sales'].replace(0.0, np.nan)
CRSP_del_delgm_delsale['delta_sales'] = CRSP_del_delgm_delsale.groupby('permno')[['delta_sales']].ffill()
CRSP_del_delgm_delsale['delta_sales'] = CRSP_del_delgm_delsale['delta_sales'].fillna(0.0)

CRSP_del_delgm_delsale['delta_delGm_minus_delSales'] = CRSP_del_delgm_delsale['delta_gm']-CRSP_del_delgm_delsale['delta_sales']
CRSP_del_delgm_delsale['delta_delGm_minus_delSales'] = CRSP_del_delgm_delsale['delta_delGm_minus_delSales'].replace([np.inf, -np.inf], np.nan)

<ipython-input-49-7c2bc1449a7f>:6: FutureWarning: The default fill_method='ffill' in DataFrameGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  CRSP_del_delgm_delsale['delta_gm'] = CRSP_del_delgm_delsale.groupby('permno')[['gross_margin']].pct_change()
<ipython-input-49-7c2bc1449a7f>:11: FutureWarning: The default fill_method='ffill' in DataFrameGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  CRSP_del_delgm_delsale['delta_sales'] = CRSP_del_delgm_delsale.groupby('permno')[['saleq']].pct_change()


In [ ]:
# Delta_so
CRSP_delta_so = CRSP_del_delgm_delsale.copy()

CRSP_delta_so['log_So'] = np.log(CRSP_delta_so['cshoq'] * CRSP_delta_so['ajexq'])

CRSP_delta_so.sort_values(['permno','yyyymm'],inplace=True)
CRSP_delta_so['delta_So'] = CRSP_delta_so.groupby('permno')[['log_So']].diff()
CRSP_delta_so['delta_So'] = CRSP_delta_so['delta_So'].replace(0.0, np.nan)
CRSP_delta_so['delta_So'] = CRSP_delta_so.groupby('permno')[['delta_So']].ffill()
CRSP_delta_so['delta_So'] = CRSP_delta_so['delta_So'].fillna(0.0)

# Handling division by 0
CRSP_delta_so['delta_So'] = CRSP_delta_so['delta_So'].replace([np.inf, -np.inf], np.nan)

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
#Delta_shrout
CRSP_delta_shrout = CRSP_delta_so.copy()

CRSP_delta_shrout.sort_values(['permno','yyyymm'],inplace=True)
CRSP_delta_shrout['delta_shrout'] = CRSP_delta_shrout.groupby('permno')[['lag_shrout']].pct_change()
CRSP_delta_shrout['delta_shrout'] = CRSP_delta_shrout['delta_shrout'].replace(0.0, np.nan)
CRSP_delta_shrout['delta_shrout'] = CRSP_delta_shrout.groupby('permno')[['delta_shrout']].ffill()
CRSP_delta_shrout['delta_shrout'] = CRSP_delta_shrout['delta_shrout'].fillna(0.0)

CRSP_delta_shrout['delta_shrout'] = CRSP_delta_shrout['delta_shrout'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# Delta_PI2A
CRSP_delta_pi2a = CRSP_delta_shrout.copy()

CRSP_delta_pi2a.sort_values(['permno','yyyymm'],inplace=True)

CRSP_delta_pi2a['ppe_and_inv'] = CRSP_delta_pi2a['ppegtq'] + CRSP_delta_pi2a['invtq']
CRSP_delta_pi2a['delta_ppe_and_inv'] = CRSP_delta_pi2a.groupby('permno')[['ppe_and_inv']].diff()
CRSP_delta_pi2a['delta_ppe_and_inv'] = CRSP_delta_pi2a['delta_ppe_and_inv'].replace(0.0, np.nan)
CRSP_delta_pi2a['delta_ppe_and_inv'] = CRSP_delta_pi2a.groupby('permno')[['delta_ppe_and_inv']].ffill()
CRSP_delta_pi2a['delta_ppe_and_inv'] = CRSP_delta_pi2a['delta_ppe_and_inv'].fillna(0.0)

CRSP_delta_pi2a['delta_PI2A'] = CRSP_delta_pi2a['delta_ppe_and_inv'] / CRSP_delta_pi2a['lag_atq']
CRSP_delta_pi2a['delta_PI2A'] = CRSP_delta_pi2a['delta_PI2A'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# DTO
dto = CRSP_daily_with_rf.copy()

nas_vol =dto.loc[dto['primaryexch']=='N','dlyvol'] * (1-0.38)
dto.loc[dto['primaryexch']=='N','dlyvol'] = nas_vol
dto['dly_turnover'] = dto['dlyvol'] / dto['shrout']

dto.sort_values(['yyyymmdd','permno'],inplace=True)
dto_mkt = dto.groupby(['yyyymmdd'])[['dly_turnover']].mean().reset_index()
dto = pd.merge(dto, dto_mkt, on=['yyyymmdd'],how='left',suffixes=('','_mkt'))
dto['dto_raw'] = dto['dly_turnover'] - dto['dly_turnover_mkt']

dto.sort_values(['permno','yyyymmdd'],inplace=True)
dto['DTO_median'] = dto.groupby(['permno'])[['dto_raw']].rolling(window=180).median().reset_index(drop=True)
dto['DTO'] = dto['dto_raw'] - dto['DTO_median']

dto['yyyymmdd'] =  pd.to_datetime(dto['yyyymmdd'].astype(str), format='%Y%m%d')
dto.sort_values(['permno','yyyymmdd'],inplace=True)
dto.set_index('yyyymmdd', inplace=True)

dto_monthly = dto.groupby('permno').resample('M').last().drop(columns=['permno']).reset_index()
dto_monthly['yyyymmdd'] = dto_monthly['yyyymmdd'].dt.strftime('%Y%m%d').astype(int)
dto_monthly['yyyymm'] = dto_monthly['yyyymmdd'] // 100

dto_monthly.sort_values(['yyyymm','permno'],inplace=True)
CRSP_delta_pi2a.sort_values(['yyyymm_l_mon','permno'],inplace=True)
CRSP_dto = pd.merge_asof(CRSP_delta_pi2a,dto_monthly[['yyyymm','permno','DTO']],by='permno',
                             left_on='yyyymm_l_mon',right_on='yyyymm',suffixes=('','_DTO'),direction='backward')

CRSP_dto['DTO'] = CRSP_dto['DTO'].replace([np.inf, -np.inf], np.nan)

In [ ]:
CRSP_dto.to_csv('dto_finished.csv')

In [ ]:
# E2P
CRSP_E2P = CRSP_dto.copy()

CRSP_E2P['E2P'] = CRSP_E2P['ibq'] / (CRSP_E2P['lag_shrout']*CRSP_E2P['lag_mthprc'])
CRSP_E2P['E2P'] = CRSP_E2P['E2P'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# EPS
CRSP_EPS = CRSP_E2P.copy()

CRSP_EPS['EPS'] = CRSP_EPS['ibq'] / CRSP_EPS['lag_shrout']
CRSP_EPS['EPS'] = CRSP_EPS['EPS'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# Free CF
CRSP_freeCF = CRSP_EPS.copy()

CRSP_freeCF['Free CF'] = (CRSP_freeCF['niq']+CRSP_freeCF['dpq']-CRSP_freeCF['wcapch']-CRSP_freeCF['capx']) / CRSP_freeCF['BV']
CRSP_freeCF['Free CF'] = CRSP_freeCF['Free CF'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# Idio vol
idio_vol= pd.read_csv('idio_vol.csv')
idio_vol.columns = idio_vol.columns.str.lower()
idio_vol['ivol'] = idio_vol['ivol'].str.rstrip('%').astype('float') / 100.0
idio_vol.sort_values(['permno','date'],inplace=True)

idio_vol['yyyymmdd'] =  pd.to_datetime(idio_vol['date'].astype(str), format='%Y%m%d')
idio_vol.sort_values(['permno','yyyymmdd'],inplace=True)
idio_vol.set_index('yyyymmdd', inplace=True)

resample_idio_vol = idio_vol.groupby('permno').resample('M').last().drop(columns=['permno'])
resample_idio_vol.reset_index(inplace=True)
resample_idio_vol['yyyymmdd'] = resample_idio_vol['yyyymmdd'].dt.strftime('%Y%m%d').astype(int)
resample_idio_vol['yyyymm'] = resample_idio_vol['yyyymmdd'] // 100

resample_idio_vol.sort_values(['yyyymm','permno'],inplace=True)
CRSP_freeCF.sort_values(['yyyymm_l_mon','permno'],inplace=True)
CRSP_idio_vol = pd.merge_asof(CRSP_freeCF,resample_idio_vol[['yyyymm','permno','ivol']],by='permno',
                             left_on='yyyymm_l_mon',right_on='yyyymm',suffixes=('','_idio_vol'),direction='backward')
CRSP_idio_vol['Idio vol'] = CRSP_idio_vol['ivol']
CRSP_idio_vol['Idio vol'] = CRSP_idio_vol['Idio vol'].replace([np.inf, -np.inf], np.nan)

In [ ]:
CRSP_idio_vol.to_csv('idio_vol_finished.csv')

In [ ]:
# Investment
CRSP_investment = CRSP_idio_vol.copy()

CRSP_investment.sort_values(['permno','yyyymm'],inplace=True)
CRSP_investment['Investment'] = CRSP_investment.groupby('permno')[['at']].pct_change()
CRSP_investment['Investment'] = CRSP_investment['Investment'].replace(0.0, np.nan)
CRSP_investment['Investment'] = CRSP_investment.groupby('permno')[['Investment']].ffill()
CRSP_investment['Investment'] = CRSP_investment['Investment'].fillna(0.0)

CRSP_investment['Investment'] = CRSP_investment['Investment'].replace([np.inf, -np.inf], np.nan)

<ipython-input-60-f0f90588ae9d>:5: FutureWarning: The default fill_method='ffill' in DataFrameGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  CRSP_investment['Investment'] = CRSP_investment.groupby('permno')[['at']].pct_change()


In [ ]:
# IPM
CRSP_ipm = CRSP_investment.copy()

CRSP_ipm['IPM'] = CRSP_ipm['piq'] / CRSP_ipm['saleq']
CRSP_ipm['IPM'] = CRSP_ipm['IPM'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# IVC
CRSP_ivc = CRSP_ipm.copy()

CRSP_ivc.sort_values(['permno','yyyymm'],inplace=True)

CRSP_ivc['delta_invt'] = CRSP_ivc.groupby('permno')[['invtq']].diff()
CRSP_ivc['delta_invt'] = CRSP_ivc['delta_invt'].replace(0.0, np.nan)
CRSP_ivc['delta_invt'] = CRSP_ivc.groupby('permno')[['delta_invt']].ffill()
CRSP_ivc['delta_invt'] = CRSP_ivc['delta_invt'].fillna(0.0)

CRSP_ivc['avg_at'] = (CRSP_ivc['atq'] + CRSP_ivc['lag_atq'])/2
CRSP_ivc['IVC'] = CRSP_ivc['delta_invt'] / CRSP_ivc['avg_at']
CRSP_ivc['IVC'] = CRSP_ivc['IVC'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# Lev
CRSP_lev = CRSP_ivc.copy()
CRSP_lev['Lev'] = (CRSP_lev['dlttq']+CRSP_lev['dlcq']) / (CRSP_lev['dlttq']+CRSP_lev['dlcq']+CRSP_lev['seqq'])
CRSP_lev['Lev'] = CRSP_lev['Lev'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# LDP
CRSP_ldp = CRSP_lev.copy()
ldp_vars = ['mthret','mthretx','mthprc','lag_mthprc']
CRSP_ldp = impute_missing_var(CRSP_ldp,
                              method='xsection',
                              compute_variables=ldp_vars)

CRSP_ldp = create_one_more_lag(CRSP_ldp,ldp_vars)

CRSP_ldp.sort_values(['permno','yyyymm'],inplace=True)
CRSP_ldp['mon_div'] = (CRSP_ldp['lag_mthret'] - CRSP_ldp['lag_mthretx']) * CRSP_ldp['lag_lag_mthprc']
CRSP_ldp['a_div'] = CRSP_ldp.groupby('permno')[['mon_div']].rolling(window=12).sum().reset_index(level=0, drop=True)
CRSP_ldp['LDP'] = CRSP_ldp['a_div'] / CRSP_ldp['lag_mthprc']
CRSP_ldp['LDP'] = CRSP_ldp['LDP'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# LME
CRSP_lme = CRSP_ldp.copy()
CRSP_lme['LME'] = CRSP_lme['lag_shrout'] * CRSP_lme['lag_mthprc']
CRSP_lme['LME'] = CRSP_lme['LME'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# LME_adj
CRSP_lme_adj = CRSP_lme.copy()

CRSP_lme_adj.sort_values(['yyyymm','sic2'],inplace=True)
lme_mean = CRSP_lme_adj.groupby(['yyyymm','sic2'],as_index=False)['LME'].mean()
lme_adj_merged = pd.merge(CRSP_lme_adj, lme_mean, on=['yyyymm','sic2'],how='left',suffixes=('', '_mean'))
lme_adj_merged['LME_adj'] = lme_adj_merged['LME'] - lme_adj_merged['LME_mean']
lme_adj_merged.sort_values(['permno','yyyymm'],inplace=True)

lme_adj_merged['LME_adj'] = lme_adj_merged['LME_adj'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# LTurnover
CRSP_lturnover = lme_adj_merged.copy()
lturnover_vars = ['mthvol','shrout']
CRSP_lturnover = impute_missing_var(CRSP_lturnover,
                                    method='xsection',
                                    compute_variables=lturnover_vars)
CRSP_lturnover = create_one_more_lag(CRSP_lturnover,lturnover_vars)

CRSP_lturnover['LTurnover'] = CRSP_lturnover['lag_mthvol'] / CRSP_lturnover['lag_shrout']
CRSP_lturnover['LTurnover'] = CRSP_lturnover['LTurnover'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# NOA
CRSP_noa = CRSP_lturnover.copy()
CRSP_noa = impute_missing_var(CRSP_noa, method='zero', compute_variables=['ivaoq'])

operating_assets = CRSP_noa['atq']-CRSP_noa['cheq']-CRSP_noa['ivaoq']
operating_liabs = CRSP_noa['atq']-CRSP_noa['dlcq']-CRSP_noa['dlttq']-CRSP_noa['mibq']-CRSP_noa['pstkq']-CRSP_noa['ceqq']
CRSP_noa['NOA'] = (operating_assets - operating_liabs) / CRSP_noa['lag_atq']
CRSP_noa['NOA'] = CRSP_noa['NOA'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# NOP
CRSP_nop = CRSP_noa.copy()
CRSP_nop['NOP'] = (CRSP_nop['dvc']+CRSP_nop['prstkc']-CRSP_nop['sstk']) / (CRSP_nop['lag_shrout']*CRSP_nop['lag_mthprc'])
CRSP_nop['NOP'] = CRSP_nop['NOP'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# O2P
CRSP_o2p = CRSP_nop.copy()
CRSP_o2p.sort_values(['permno','yyyymm'],inplace=True)

CRSP_o2p['delta_pstkrv'] = CRSP_o2p.groupby('permno')[['pstkrv']].diff()
CRSP_o2p['delta_pstkrv'] = CRSP_o2p['delta_pstkrv'].replace(0.0, np.nan)
CRSP_o2p['delta_pstkrv'] = CRSP_o2p.groupby('permno')[['delta_pstkrv']].ffill()
CRSP_o2p['delta_pstkrv'] = CRSP_o2p['delta_pstkrv'].fillna(0.0)

CRSP_o2p['O2P'] = (CRSP_o2p['dvc']+CRSP_o2p['prstkc']-CRSP_o2p['delta_pstkrv']) / (CRSP_o2p['lag_shrout']*CRSP_o2p['lag_mthprc'])
CRSP_o2p['O2P'] = CRSP_o2p['O2P'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# OA
CRSP_oa = CRSP_o2p.copy()

CRSP_oa['non_cash_CA'] = CRSP_oa['actq']-CRSP_oa['cheq']
CRSP_oa['non_cash_WC'] = CRSP_oa['non_cash_CA']-(CRSP_oa['lctq']+CRSP_oa['dlcq']+CRSP_oa['txpq'])

CRSP_oa.sort_values(['permno','yyyymm'],inplace=True)

CRSP_oa['non_cash_WC_delta'] = CRSP_oa.groupby('permno')[['non_cash_WC']].diff()
CRSP_oa['non_cash_WC_delta'] = CRSP_oa['non_cash_WC_delta'].replace(0.0, np.nan)
CRSP_oa['non_cash_WC_delta'] = CRSP_oa.groupby('permno')[['non_cash_WC_delta']].ffill()
CRSP_oa['non_cash_WC_delta'] = CRSP_oa['non_cash_WC_delta'].fillna(0.0)

CRSP_oa['OA'] = (CRSP_oa['non_cash_WC_delta']-CRSP_oa['dpq']) / CRSP_oa['lag_atq']
CRSP_oa['OA'] = CRSP_oa['OA'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# OL
CRSP_ol = CRSP_oa.copy()
CRSP_ol['OL'] = (CRSP_ol['cogsq']+CRSP_ol['xsgaq']) / CRSP_ol['atq']
CRSP_E2P['E2P'] = CRSP_E2P['E2P'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# PCM
CRSP_pcm = CRSP_ol.copy()
CRSP_pcm['PCM'] = (CRSP_pcm['saleq']-CRSP_pcm['cogsq']) / CRSP_pcm['saleq']
CRSP_pcm['PCM'] = CRSP_pcm['PCM'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# PM
CRSP_pm = CRSP_pcm.copy()
CRSP_pm['PM'] = CRSP_pm['oiadpq'] / CRSP_pm['saleq']
CRSP_pm['PM'] = CRSP_pm['PM'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# PM_adj
CRSP_pm_adj = CRSP_pm.copy()
CRSP_pm_adj.sort_values(['yyyymm','sic2'],inplace=True)
pm_mean = CRSP_pm_adj.groupby(['yyyymm','sic2'],as_index=False)['PM'].mean()
pm_adj_merged = pd.merge(CRSP_pm_adj, pm_mean, on=['yyyymm','sic2'],how='left',suffixes=('', '_mean'))

pm_adj_merged.sort_values(['permno','yyyymm'],inplace=True)
pm_adj_merged['PM_adj'] = pm_adj_merged['PM'] - pm_adj_merged['PM_mean']
pm_adj_merged['PM_adj'] = pm_adj_merged['PM_adj'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# Prof
CRSP_prof = pm_adj_merged.copy()
CRSP_prof.sort_values(['permno','yyyymm'],inplace=True)

CRSP_prof['Prof'] = CRSP_prof['gp'] / CRSP_prof['BV']
CRSP_prof['Prof'] = CRSP_prof['Prof'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# Q
CRSP_tobin_q = CRSP_prof.copy()
CRSP_tobin_q.sort_values(['permno','yyyymm'],inplace=True)

CRSP_tobin_q['Q'] = (CRSP_tobin_q['atq']+CRSP_tobin_q['lag_shrout']*CRSP_tobin_q['lag_mthprc']-CRSP_tobin_q['ceqq']-CRSP_tobin_q['txdbq']) / CRSP_tobin_q['atq']
CRSP_tobin_q['Q'] = CRSP_tobin_q['Q'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# Rel to High
CRSP_rel2high = CRSP_tobin_q.copy()
CRSP_rel2high.sort_values(['permno','yyyymm'],inplace=True)

CRSP_rel2high['52week_high'] = CRSP_rel2high.groupby('permno')[['lag_mthprc']].rolling(window=12).max().reset_index(level=0, drop=True)
CRSP_rel2high['Rel to High'] = CRSP_rel2high['lag_mthprc'] / CRSP_rel2high['52week_high']
CRSP_rel2high['Rel to High'] = CRSP_rel2high['Rel to High'].replace([np.inf, -np.inf], np.nan)

In [ ]:
CRSP_rel2high.to_csv('rel2high_finished.csv')

In [ ]:
# Ret_max
ret_max = CRSP_daily_with_rf.copy().reset_index()
ret_max['yyyymm'] = ret_max['yyyymmdd'] // 100
ret_max.sort_values(['permno','yyyymm'],inplace=True)

ret_max_grouped = ret_max.groupby(['permno','yyyymm'])[['dlyret']].max().reset_index()
ret_max_grouped.sort_values(['yyyymm','permno'],inplace=True)
CRSP_rel2high.sort_values(['yyyymm_l_mon','permno'],inplace=True)
ret_max_merged = pd.merge_asof(CRSP_rel2high, ret_max_grouped[['permno','yyyymm','dlyret']],by='permno',
                               left_on='yyyymm_l_mon',right_on='yyyymm',suffixes=('','_retmax'),direction='backward')
ret_max_merged['Ret_max'] = ret_max_merged['dlyret']
ret_max_merged['Ret_max'] = ret_max_merged['Ret_max'].replace([np.inf, -np.inf], np.nan)

In [ ]:
ret_max_merged.to_csv('ret_max_finished.csv')

In [ ]:
# RNA
CRSP_rna = ret_max_merged.copy()
rna_vars = ['oiadpq','atq','lag_atq','cheq','ivaoq','dlcq','dlttq','mibq','pstkq','ceqq']
CRSP_rna = create_one_more_lag(CRSP_rna, rna_vars)

# lagged_operating_assets = lagged_AT - lagged_CHE - lagged_IVAO
lag_operating_assets = CRSP_rna['lag_atq']-CRSP_rna['lag_cheq']-CRSP_rna['lag_ivaoq']
# lagged_operating_liabilities = lagged_AT - lagged_DLC - lagged_DLTT - lagged_MIB - lagged_PSTK - lagged_CEQ
lag_operating_liabs = CRSP_rna['lag_atq']-CRSP_rna['lag_dlcq']-CRSP_rna['lag_dlttq']-CRSP_rna['lag_mibq']-CRSP_rna['lag_pstkq']-CRSP_rna['lag_ceqq']

CRSP_rna['RNA'] = CRSP_rna['oiadpq'] / (lag_operating_assets-lag_operating_liabs)
CRSP_rna['RNA'] = CRSP_rna['RNA'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# ROA
CRSP_roa = CRSP_rna.copy()
CRSP_roa['ROA'] = CRSP_roa['ibq'] / CRSP_roa['lag_atq']
CRSP_roa['ROA'] = CRSP_roa['ROA'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# ROC
CRSP_roc = CRSP_roa.copy()
CRSP_roc['ROC'] = (CRSP_roc['mkvaltq']+CRSP_roc['dlttq']-CRSP_roc['atq']) / CRSP_roc['cheq']
CRSP_roc['ROC'] = CRSP_roc['ROC'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# ROE
CRSP_roe = CRSP_roc.copy()
roe_vars = ['ibq','BV']
CRSP_roe = create_one_more_lag(CRSP_roe,roe_vars)

CRSP_roe['ROE'] = CRSP_roe['ibq'] / CRSP_roe['lag_BV']
CRSP_roe['ROE'] = CRSP_roe['ROE'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# ROIC
CRSP_roic = CRSP_roe.copy()
CRSP_roic['ROIC'] = (CRSP_roic['ebit']-CRSP_roic['nopiq']) / (CRSP_roic['ceqq']+CRSP_roic['ltq']+CRSP_roic['cheq'])
CRSP_roic['ROIC'] = CRSP_roic['ROIC'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# r12_2
CRSP_r_12_2 = CRSP_roic.copy()
def rolling_cum_return_12(x):
    return (x + 1).rolling(window=12).apply(lambda y: y.prod()) - 1

def rolling_cum_return_2(x):
    return (x + 1).rolling(window=2).apply(lambda y: y.prod()) - 1

CRSP_r_12_2.sort_values(['permno','yyyymm'],inplace=True)
CRSP_r_12_2['r_12'] = CRSP_r_12_2.groupby('permno')[['lag_mthret']].apply(rolling_cum_return_12).reset_index(level=0, drop=True)
CRSP_r_12_2['r_2'] = CRSP_r_12_2.groupby('permno')[['lag_mthret']].apply(rolling_cum_return_2).reset_index(level=0, drop=True)
CRSP_r_12_2['r_12_2'] = CRSP_r_12_2['r_12'] / CRSP_r_12_2['r_2']
CRSP_r_12_2['r_12_2'] = CRSP_r_12_2['r_12_2'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# r12_7
CRSP_r_12_7 = CRSP_r_12_2.copy()
def rolling_cum_return_7(x):
    return (x + 1).rolling(window=7).apply(lambda y: y.prod()) - 1

CRSP_r_12_7.sort_values(['permno','yyyymm'],inplace=True)
CRSP_r_12_7['r_7'] = CRSP_r_12_7.groupby('permno')[['lag_mthret']].apply(rolling_cum_return_7).reset_index(level=0, drop=True)
CRSP_r_12_7['r_12_7'] = CRSP_r_12_7['r_12'] / CRSP_r_12_7['r_7']
CRSP_r_12_7['r_12_7'] = CRSP_r_12_7['r_12_7'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# r6_2
CRSP_r_6_2 = CRSP_r_12_7.copy()
def rolling_cum_return_6(x):
    return (x + 1).rolling(window=6).apply(lambda y: y.prod()) - 1

CRSP_r_6_2.sort_values(['permno','yyyymm'],inplace=True)
CRSP_r_6_2['r_6'] = CRSP_r_6_2.groupby('permno')[['lag_mthret']].apply(rolling_cum_return_6).reset_index(level=0, drop=True)
CRSP_r_6_2['r_6_2'] = CRSP_r_6_2['r_6'] / CRSP_r_6_2['r_2']
CRSP_r_6_2['r_6_2'] = CRSP_r_6_2['r_6_2'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# r2_1
CRSP_r_2_1 = CRSP_r_6_2.copy()
CRSP_r_2_1.sort_values(['permno','yyyymm'],inplace=True)

CRSP_r_2_1['r_2_1'] = CRSP_r_2_1['lag_mthret']
CRSP_r_2_1['r_2_1'] = CRSP_r_2_1['r_2_1'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# r36_13
CRSP_r_36_13 = CRSP_r_2_1.copy()
def rolling_cum_return_36(x):
    return (x + 1).rolling(window=36).apply(lambda y: y.prod()) - 1

def rolling_cum_return_13(x):
    return (x + 1).rolling(window=13).apply(lambda y: y.prod()) - 1

CRSP_r_36_13.sort_values(['permno','yyyymm'],inplace=True)
CRSP_r_36_13['r_36'] = CRSP_r_36_13.groupby('permno')[['lag_mthret']].apply(rolling_cum_return_36).reset_index(level=0, drop=True)
CRSP_r_36_13['r_13'] = CRSP_r_36_13.groupby('permno')[['lag_mthret']].apply(rolling_cum_return_13).reset_index(level=0, drop=True)
CRSP_r_36_13['r_36_13'] = CRSP_r_36_13['r_36'] / CRSP_r_36_13['r_13']
CRSP_r_36_13['r_36_13'] = CRSP_r_36_13['r_36_13'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# S2C
CRSP_s2c = CRSP_r_36_13.copy()
CRSP_s2c.sort_values(['permno','yyyymm'],inplace=True)

CRSP_s2c['S2C'] = CRSP_s2c['saleq'] / CRSP_s2c['cheq']
CRSP_s2c['S2C'] = CRSP_s2c['S2C'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# S2P
CRSP_s2p = CRSP_s2c.copy()
CRSP_s2p.sort_values(['permno','yyyymm'],inplace=True)

CRSP_s2p['S2P'] = CRSP_s2p['saleq'] / (CRSP_s2p['lag_shrout']*CRSP_s2p['lag_mthprc'])
CRSP_s2p['S2P'] = CRSP_s2p['S2P'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# Sales_g
CRSP_sales_g = CRSP_s2p.copy()
CRSP_sales_g.sort_values(['permno','yyyymm'],inplace=True)

CRSP_sales_g['Sales_g'] = CRSP_sales_g.groupby('permno')[['sale']].pct_change()
CRSP_sales_g['Sales_g'] = CRSP_sales_g['Sales_g'].replace(0.0, np.nan)
CRSP_sales_g['Sales_g'] = CRSP_sales_g.groupby('permno')[['Sales_g']].ffill()
CRSP_sales_g['Sales_g'] = CRSP_sales_g['Sales_g'].fillna(0.0)
CRSP_sales_g['Sales_g'] = CRSP_sales_g['Sales_g'].replace([np.inf, -np.inf], np.nan)

<ipython-input-94-a6add1ce2ca3>:5: FutureWarning: The default fill_method='ffill' in DataFrameGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  CRSP_sales_g['Sales_g'] = CRSP_sales_g.groupby('permno')[['sale']].pct_change()


In [ ]:
# SAT
CRSP_sat= CRSP_sales_g.copy()
CRSP_sat.sort_values(['permno','yyyymm'],inplace=True)

CRSP_sat['SAT'] = CRSP_sat['saleq'] / CRSP_sat['atq']
CRSP_sat['SAT'] = CRSP_sat['SAT'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# SAT_adj
CRSP_sat_adj = CRSP_sat.copy()

CRSP_sat_adj.sort_values(['yyyymm','sic2'],inplace=True)
sat_mean = CRSP_sat_adj.groupby(['yyyymm','sic2'],as_index=False)['SAT'].mean()
sat_adj_merged = pd.merge(CRSP_sat_adj, sat_mean, on=['yyyymm','sic2'],how='left',suffixes=('', '_mean'))

sat_adj_merged['SAT_adj'] = sat_adj_merged['SAT'] - sat_adj_merged['SAT_mean']
sat_adj_merged.sort_values(['permno','yyyymm'],inplace=True)
sat_adj_merged['SAT_adj'] = sat_adj_merged['SAT_adj'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# SGA2S
CRSP_sga2s= sat_adj_merged.copy()
CRSP_sga2s.sort_values(['permno','yyyymm'],inplace=True)

CRSP_sga2s['SGA2S'] = CRSP_sga2s['xsgaq'] / CRSP_sga2s['saleq']
CRSP_sga2s['SGA2S'] = CRSP_sga2s['SGA2S'].replace([np.inf, -np.inf], np.nan)

In [ ]:
CRSP_sga2s.to_csv('sga2s_finished.csv')

In [ ]:
# Spread
spread = CRSP_daily_with_rf.copy().reset_index()
spread['yyyymm'] = spread['yyyymmdd'] // 100
spread.sort_values(['permno','yyyymm'],inplace=True)

spread['bid-ask spread'] = spread['dlyask'] - spread['dlybid']
spread = spread.groupby(['permno','yyyymm'])[['bid-ask spread']].mean().reset_index()

spread.sort_values(['yyyymm','permno'],inplace=True)
CRSP_sga2s.sort_values(['yyyymm_l_mon','permno'],inplace=True)
spread_mean_merged = pd.merge_asof(CRSP_sga2s, spread[['permno','yyyymm','bid-ask spread']],by='permno',
                              left_on='yyyymm_l_mon',right_on='yyyymm',suffixes=('','_spread'),direction='backward')

spread_mean_merged['Spread'] = spread_mean_merged['bid-ask spread']
spread_mean_merged.sort_values(['permno','yyyymm'],inplace=True)
spread_mean_merged['Spread'] = spread_mean_merged['Spread'].replace([np.inf, -np.inf], np.nan)

In [ ]:
spread_mean_merged.to_csv('spread_finished.csv')

In [ ]:
spread_mean_merged = pd.read_csv('spread_finished.csv')

<ipython-input-7-64dafc406462>:1: DtypeWarning: Columns (2,3,4,5,23,33,80) have mixed types. Specify dtype option on import or set low_memory=False.
  spread_mean_merged = pd.read_csv('spread_finished.csv')


In [ ]:
spread_mean_merged.iloc[:,[2,3,4,5,23,33,80]] = spread_mean_merged.iloc[:,[2,3,4,5,23,33,80]].astype(str)

In [ ]:
# Std_turnover
std_turnover = CRSP_daily_with_rf.copy().reset_index()
std_turnover.sort_values(['permno','yyyymmdd'],inplace=True)
std_turnover['dly_turnover'] = std_turnover['dlyvol'] / std_turnover['shrout']

def std_turnover_rolling_resid_std(group):
  # Check if the group has enough data points
  if len(group) < 21 or all(group['dly_turnover'].isna()):
      return group['dly_turnover']

  resid_std = []
  for x in group.rolling(21):
    resid = x['dly_turnover'] - x['dlyturnover_mean'].iloc[-1]
    resid_std.append(resid.std())

  return pd.Series(resid_std, index=group.index)

std_turnover['dlyturnover_mean'] = std_turnover.groupby('permno')[['dly_turnover']].rolling(21).mean().reset_index(drop=True)
std_turnover['Std_turnover'] = std_turnover.groupby('permno').apply(std_turnover_rolling_resid_std).reset_index(drop=True)
std_turnover['Std_turnover'] = std_turnover['Std_turnover'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# Std_turnover (continued)
std_turnover['yyyymmdd'] =  pd.to_datetime(std_turnover['yyyymmdd'].astype(str), format='%Y%m%d')
std_turnover.sort_values(['permno','yyyymmdd'],inplace=True)
std_turnover.set_index(['yyyymmdd'],inplace=True)
std_turnover_mon = std_turnover.groupby('permno').resample('M').last().drop(columns=['permno']).reset_index()
std_turnover_mon['yyyymm'] = (std_turnover_mon['yyyymmdd'].dt.strftime('%Y%m%d').astype(int)) // 100

std_turnover_mon.sort_values(['yyyymm','permno'],inplace=True)
spread_mean_merged.sort_values(['yyyymm_l_mon','permno'],inplace=True)
std_turnover_merged = pd.merge_asof(spread_mean_merged, std_turnover_mon[['permno','yyyymm','Std_turnover']], by='permno',
                               left_on='yyyymm_l_mon', right_on='yyyymm', suffixes=('','_std_turnover'), direction='backward')

In [ ]:
std_turnover_merged.to_csv('std_turnover_merged.csv')

In [ ]:
std_turnover_merged = pd.read_csv('std_turnover_merged.csv')
std_turnover_merged.iloc[:,[3,4,5,6,24,34,81]] = std_turnover_merged.iloc[:,[3,4,5,6,24,34,81]].astype(str)

<ipython-input-20-97ff1d325f95>:1: DtypeWarning: Columns (3,4,5,6,24,34,81) have mixed types. Specify dtype option on import or set low_memory=False.
  std_turnover_merged = pd.read_csv('std_turnover_merged.csv')


In [ ]:
# Std_volume
std_volume = CRSP_daily_with_rf.copy().reset_index()
std_volume.sort_values(['permno','yyyymmdd'],inplace=True)

def std_vol_rolling_resid_std(group):
  # Check if the group has enough data points
  if len(group) < 21 or all(group['dlyvol'].isna()):
      return group['dlyvol']

  resid_std = []
  for x in group.rolling(21):
    resid = x['dlyvol'] - x['dlyvol_mean'].iloc[-1]
    resid_std.append(resid.std())

  return pd.Series(resid_std, index=group.index)

std_volume['dlyvol_mean'] = std_volume.groupby('permno')[['dlyvol']].rolling(21).mean().reset_index(drop=True)
std_volume['Std_volume'] = std_volume.groupby('permno').apply(std_vol_rolling_resid_std).reset_index(drop=True)
std_volume['Std_volume'] = std_volume['Std_volume'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# Std_volume (continued)
std_volume['yyyymmdd'] =  pd.to_datetime(std_volume['yyyymmdd'].astype(str), format='%Y%m%d')
std_volume.sort_values(['permno','yyyymmdd'],inplace=True)
std_volume.set_index(['yyyymmdd'],inplace=True)
std_volume_mon = std_volume.groupby('permno').resample('M').last().drop(columns=['permno']).reset_index()
std_volume_mon['yyyymm'] = (std_volume_mon['yyyymmdd'].dt.strftime('%Y%m%d').astype(int)) // 100

std_volume_mon.sort_values(['yyyymm','permno'],inplace=True)
std_turnover_merged.sort_values(['yyyymm_l_mon','permno'],inplace=True)
std_volume_merged = pd.merge_asof(std_turnover_merged, std_volume_mon[['permno','yyyymm','Std_volume']], by='permno',
                               left_on='yyyymm_l_mon',right_on='yyyymm',suffixes=('','_std_vol'), direction='backward')

In [ ]:
std_volume_merged.to_csv('std_volume_merged.csv')

In [ ]:
std_volume_merged = pd.read_csv('std_volume_merged.csv')
std_volume_merged.iloc[:,[4,5,6,7,25,35,82]] = std_volume_merged.iloc[:,[4,5,6,7,25,35,82]].astype(str)

<ipython-input-38-c06df525561e>:1: DtypeWarning: Columns (4,5,6,7,25,35,82) have mixed types. Specify dtype option on import or set low_memory=False.
  std_volume_merged = pd.read_csv('std_volume_merged.csv')


In [ ]:
# SUV
# Define a function to perform the rolling regression
def suv_rolling_regression(group):
    # Check if the group has enough data points
    if len(group) < 21 or all(group['abs_ret'].isna()) or all(group['dlyvol'].isna()):
        return pd.Series([0] * len(group))

    rolling_model = RollingOLS(group['dlyvol'], sm.add_constant(group[['abs_ret']]), window=21, min_nobs=15).fit()
    return rolling_model.params

def rolling_resid_std(group):
  # Check if the group has enough data points
  if len(group) < 21 or all(group['abs_ret'].isna()) or all(group['dlyvol'].isna()):
      return group['dlyvol']
  resid_std = []
  for x in group.rolling(21):
    resid = x['dlyvol'] - x['suv_const'].iloc[-1] - x['reg_abs_ret'].iloc[-1] * x['abs_ret']
    resid_std.append(resid.std())

  return pd.Series(resid_std, index=group.index)


suv = CRSP_daily_with_rf.copy()
suv['abs_ret'] = suv['dlyret'].abs()
suv.sort_values(['permno','yyyymmdd'],inplace=True)
suv.reset_index(drop=True,inplace=True)

reg_params = suv.groupby('permno',group_keys=False).apply(suv_rolling_regression)
suv.reset_index(drop=True,inplace=True)
reg_params.reset_index(drop=True,inplace=True)

# The warnings below is due to zero or close-to-zero SSR. It's a fitting issue rather than a calculation/data issue.
# Therefore, we can ignore the warning for now.

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/rolling.py:272: RuntimeWarning: divide by zero encountered in log
  llf = -np.log(ssr) * nobs2  # concentrated likelihood
/usr/local/lib/python3.10/dist-packages/statsmodels/regression/rolling.py:272: RuntimeWarning: divide by zero encountered in log
  llf = -np.log(ssr) * nobs2  # concentrated likelihood
/usr/local/lib/python3.10/dist-packages/statsmodels/regression/rolling.py:272: RuntimeWarning: divide by zero encountered in log
  llf = -np.log(ssr) * nobs2  # concentrated likelihood
/usr/local/lib/python3.10/dist-packages/statsmodels/regression/rolling.py:272: RuntimeWarning: divide by zero encountered in log
  llf = -np.log(ssr) * nobs2  # concentrated likelihood
/usr/local/lib/python3.10/dist-packages/statsmodels/regression/rolling.py:272: RuntimeWarning: divide by zero encountered in log
  llf = -np.log(ssr) * nobs2  # concentrated likelihood
/usr/local/lib/python3.10/dist-packages/statsmodels/regression/rolling.py:2

In [ ]:
# SUV (continued)
suv['suv_const']= reg_params['const']
suv['reg_abs_ret'] = reg_params['abs_ret']
suv['resid'] = suv['dlyvol'] - suv['suv_const'] - suv['reg_abs_ret'] * suv['abs_ret']
suv['std'] = suv.groupby('permno',group_keys=False)[['dlyvol','abs_ret','suv_const','reg_abs_ret']].apply(rolling_resid_std)
suv['SUV'] = suv['resid'] / suv['std']
suv['SUV'] = suv['SUV'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# SUV (continued)
suv['yyyymmdd'] =  pd.to_datetime(suv['yyyymmdd'].astype(str), format='%Y%m%d')
suv.sort_values(['permno','yyyymmdd'],inplace=True)
suv.set_index(['yyyymmdd'],inplace=True)
suv_monthly = suv.groupby('permno').resample('M').last().drop(columns=['permno']).reset_index()
suv_monthly['yyyymmdd'] = suv_monthly['yyyymmdd'].dt.strftime('%Y%m%d').astype(int)
suv_monthly['yyyymm'] = suv_monthly['yyyymmdd'] // 100

suv_monthly.sort_values(['yyyymm','permno'],inplace=True)
std_volume_merged.sort_values(['yyyymm_l_mon','permno'],inplace=True)
suv_merged = pd.merge_asof(std_volume_merged,suv_monthly[['permno','yyyymm','SUV']], by='permno',
                           left_on='yyyymm_l_mon',right_on='yyyymm',suffixes=('','_suv'),direction='backward')

In [ ]:
suv_merged.to_csv('suv_merged.csv')

In [ ]:
suv_merged = pd.read_csv('suv_merged.csv')
suv_merged.iloc[:,[5,6,7,8,26,36,83]] = suv_merged.iloc[:,[5,6,7,8,26,36,83]].astype(str)

<ipython-input-22-2a6d140451d3>:1: DtypeWarning: Columns (5,6,7,8,26,36,83) have mixed types. Specify dtype option on import or set low_memory=False.
  suv_merged = pd.read_csv('suv_merged.csv')


In [ ]:
# Tan
CRSP_tan = suv_merged.copy()

CRSP_tan['Tan'] = (0.715*CRSP_tan['rectq']+0.547*CRSP_tan['invtq']+0.535*CRSP_tan['ppentq']+CRSP_tan['cheq'])/CRSP_tan['atq']
CRSP_tan['Tan'] = CRSP_tan['Tan'].replace([np.inf, -np.inf], np.nan)

In [ ]:
# Total vol
total_vol = CRSP_daily_with_rf.copy().reset_index()
total_vol.sort_values(['permno','yyyymmdd'],inplace=True)
total_vol['excess_ret'] = total_vol['dlyret'] - total_vol['rf']

def total_vol_rolling_resid_std(group):
  # Check if the group has enough data points
  if len(group) < 21 or all(group['excess_ret'].isna()):
      return group['excess_ret']

  resid_std = []
  for x in group.rolling(21):
    resid = x['excess_ret'] - x['dlyexret_mean'].iloc[-1]
    resid_std.append(resid.std())

  return pd.Series(resid_std, index=group.index)

total_vol['dlyexret_mean'] = total_vol.groupby('permno')[['excess_ret']].rolling(21).mean().reset_index(drop=True)
total_vol['Total_vol'] = total_vol.groupby('permno').apply(total_vol_rolling_resid_std).reset_index(drop=True)

total_vol['yyyymmdd'] =  pd.to_datetime(total_vol['yyyymmdd'].astype(str), format='%Y%m%d')
total_vol.sort_values(['permno','yyyymmdd'],inplace=True)
total_vol.set_index(['yyyymmdd'],inplace=True)
total_vol_mon = total_vol.groupby('permno').resample('M').last().drop(columns=['permno']).reset_index()
total_vol_mon['yyyymm'] = (total_vol_mon['yyyymmdd'].dt.strftime('%Y%m%d').astype(int)) // 100

total_vol_mon.sort_values(['yyyymm','permno'],inplace=True)
CRSP_tan.sort_values(['yyyymm_l_mon','permno'],inplace=True)
total_vol_merged = pd.merge_asof(CRSP_tan, total_vol_mon[['permno','yyyymm','Total_vol']], by='permno',
                               left_on='yyyymm_l_mon',right_on='yyyymm',suffixes=('','_total_vol'),direction='backward')
total_vol_merged['Total_vol'] = total_vol_merged['Total_vol'].replace([np.inf, -np.inf], np.nan)

In [ ]:
total_vol_merged.to_csv('total_vol_merged.csv')

In [ ]:
# The calculation is finished!

# Calculated features
freyberger_features = ['AT','A2ME','AOA','ATO','BEME','BEME_adj',
            'Beta_FP','Beta_LN','Beta_daily','C','C2D','CTO','Debt2P','delta_ceq','delta_delGm_minus_delSales',
            'delta_So','delta_shrout','delta_PI2A','DTO','E2P','EPS','Free CF','Idio vol',
            'Investment','IPM','IVC','Lev','LDP','LME','LME_adj','LTurnover','NOA','NOP','O2P',
            'OA','OL','PCM','PM','PM_adj','Prof','Q','Rel to High','Ret_max','RNA','ROA','ROC',
            'ROE','ROIC','r_12_2','r_12_7','r_6_2','r_2_1','r_36_13','S2C','S2P','Sales_g','SAT',
            'SAT_adj','SGA2S','Spread','Std_turnover','Std_volume','Tan','Total_vol']

# Suppose df is your data frame that contains the features
# Final dataframe:
final_df = total_vol_merged[['yyyymm','permno','gvkey','primaryexch','mthret','sprtrn','rf','sic2']+freyberger_features]
final_df.sort_values(['yyyymm','permno'],inplace=True)
final_df = impute_missing_var(final_df, method='xsection', compute_variables=freyberger_features)
final_df.replace([np.nan], 0, inplace=True)

<ipython-input-29-b3d14a37a10a>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.sort_values(['yyyymm','permno'],inplace=True)
<ipython-input-13-b3f287127421>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['yyyy_lag1']=df['yyyymm']//100-1
<ipython-input-13-b3f287127421>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

In [ ]:
ranked_df = final_df.groupby('yyyymm')[freyberger_features].rank(method='min')  # Min rank ensures the smallest value gets rank 1
temp1_df = pd.concat([final_df[['yyyymm','permno','gvkey','primaryexch','mthret','sprtrn','rf','sic2']],ranked_df],axis=1)
temp2_df = temp1_df.groupby('yyyymm')[freyberger_features].transform(lambda x: x / (len(x) + 1))
normalized_df = pd.concat([temp1_df[['yyyymm','permno','gvkey','primaryexch','mthret','sprtrn','rf','sic2']],temp2_df],axis=1)

normalized_df.to_csv('features.csv')

print('Done.')

Done.
